<div dir="rtl" align="center">
<font face="XB Zar" size=5>
    <font face="IranNastaliq" size=5>
      به نام خدا
    </font>
    <br>
    <font size=3>
      دانشگاه صنعتی شریف - دانشکده مهندسی کامپیوتر
    </font>
    <br>
    <font color=blue size=5>
      پردازش زبان‌های طبیعی
    </font>
    <br>
    <hr/>
    <font color=red size=6>
      تمرین دوم: تشخیص کلمات غیرقانونی
    </font>
    <br>
      حدیث احمدیان، سینا عبدوس، مهسا یزدانی
    <hr>
<br>
  <div align="right">

  <div>
</div>

<div dir='rtl' style='direction:rtl:'>
<h1>تعریف مسئله</h1>
<div>ما در این تمرین می‌خواهیم به عنوان ورودی یک متن و یک لیست از کلمات غیر قانونی بگیریم و تشخیص دهیم که آیا در این متن، این کلمات غیرقانونی وجود دارند یا خیر و اگر وجود داشتند، بازه‌ی هر کلمه‌ی غیرقانونی را نیز مشخص کنیم.</div>

<h2>کلاس اصلی</h2>
<div> یک کلاس اصلی به نام DetectIlligalWords وجود دارد که تابعی به نام run، تابع اصلی آن است. این تابع ابتدا تابع preprocess و سپس تابع process را فراخوانی می‌کند.

<h4> تابع preprocess </h4>
با فراخوانی تابع run ابتدا تابع preprocess فراخوانی می‌شوند.
در تابع preprocess ابتدا normalization انجام شده و همچنین ایموجی‌های موجود در متن به متن تعریف می‌شوند. سپس روی این متن tokenization انجام می‌شود و خروجی تابع preprocess لیستی از tokenها خواهد بود.

نکته: تبدیل ایموجی به متن به این دلیل است که ممکن است حالتی مانند 🐘ترشکن ایجاد شود که در آن کلمه غیرقانونی فیلترشکن همراه با یک ایموجی نوشته شده است. برای تبدیل ایموجی به متن، از دیتاستی استفاده شده است که هر ایموجی را به عبارتی که آن را توصیف می‌کند، map شده است.

<h4> تابع process </h4>
در این تابع ابتدا تابع detect روی خود tokenها فراخوانی شده و سپس یک بار هم روی lemmaی tokenها فراخوانی می‌شود. چرا که ممکن است به طور مثال بجای خود کلمه‌ی تفنگ در متن داشته باشیم تفنگ‌ها.
تابع detect تعدادی دیگری از توابع را فراخوانی می‌کند که در نهایت هر کدام از آن‌ها حالت‌های مختلفی از به کار بردن کلمه‌ی غیرقانونی را تشخیص می‌دهند که در ادامه هر کدام از این توابع و مثال‌هایی از مواردی که تشخیص می‌دهند، توضیح داده خواهد شد.

</div>
</div>

<div dir='rtl' style='direction:rtl:'>
<hr/>
در سلول‌های بعدی، ماژول‌های مورد استفاده که در فایل requirements.txt نوشته شده‌اند، ابتدا نصب و سپس import می‌شوند.
</div>


In [ ]:
from typing_extensions import Text
!touch /content/requirments.txt

In [ ]:
%%writefile /content/requirments.txt

hazm
num2fawords

Overwriting /content/requirments.txt


In [ ]:
!pip install -r /content/requirments.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from __future__ import unicode_literals
from hazm import *
import re

<div dir='rtl' style='direction:rtl:'>
<h3> توابع استفاده شده </h3>
<h4> تابع is_meaningful_word </h4>
به منظور اینکه اشتباهی در تشخیص کلمات غیرقانونی پیش نیاید و کلمه‌ای معنی‌دار به عنوان کلمه‌ای غیرقانونی که در بین حروف آن کاراکتر‌های فارسی خاص به کار رفته است تشخیص داده نشود، پس از تشخیص حالت ابتدایی کلمات غیرقانونی (یعنی به شکل مستقیم به کار رفتن آن‌ها)، tokenهای معنی دار از لیست tokenها حذف خواهند شد.
<br>
به این منظور فایلی به نام persian_words.txt در پوشه‌ی پروژه قرار داده شده است که لیستی از کلمات معنادار فارسی است و برای چک کردن معنی‌دار بودن یا نبودن یک کلمه، چک می‌شود که آیا کلمه در این لیست هست یا خیر.
</div>


In [ ]:
# read persian words from text file and convert to a list
%cd /content
!gdown 1FGMH4pzNmN3x8WE4vGpd03F95O4pITcL

with open('/content/persian_words.txt', 'r', encoding="utf8") as f:
    persian_words = [line.strip() for line in f.read().splitlines()]

# cheks whether the token is meaningful or not
def is_meaningful_word(token):
  if token in persian_words:
      return True
  return False

/content
Downloading...
From: https://drive.google.com/uc?id=1FGMH4pzNmN3x8WE4vGpd03F95O4pITcL
To: /content/persian_words.txt
100% 25.8M/25.8M [00:00<00:00, 142MB/s] 


<div dir='rtl' style='direction:rtl:'>
<h4> تابع detect_with_star </h4>
این تابع تابعی است که حالاتی را چک می‌کند که در آن یک یا تعدادی از حروف کلمه‌ی غیرقانونی با * جایگزین شده‌اند. به عنوان مثال بجای تفنگ حالتی مانند ت**گ نوشته شده است.
</div>

In [ ]:
# examples: "تف*گ ت**گ *ف*گ نف*گ" for "تفنگ"
def detect_with_star(token, illegal_words):


  """
  To check asterisk occurrence in the word.
  Illegal and input words should have the same length

  """

  if token == '.':
    return None

  for illegal_word in illegal_words:

    if len(token) < len(illegal_word):
      return None

    pattern = re.compile(token.replace("*", "[[\u0600-\u06FF]|گ|چ|پ|ژ]"), re.IGNORECASE)
    match = re.search(pattern, illegal_word)
    if match:
      return illegal_word
    else:
      return None

# detect_with_star('نگ', ['تفنگ'])

In [ ]:
def all_formats_a(word,all,indx): 
  all.add(word)
  flag=True
  for i in range(indx,len(word)):
    if word[i]=="a" or word[i]=="A":
      flag=False
      all_formats_a(word,all,i+1)
      all_formats_a(word[:i]+word[i+1:],all,i)
  return all

  

<div dir='rtl' style='direction:rtl:'>
<h4> توابع  all_formats_o , all_formats_a </h4>
این توابع پیش پردازش های لازم برای شناسایی کلمات فینگلیش را انجام میدهند. در فارسی ما اعراب کلمات را نمینویسیم اما در انگلیسی با کاراکترهایی مانند o و a و e اعراب ها را نیز مینویسیم، حال آنکه بعضی از این کاراکتر ها (مثل a) میتواند هم برای اعراب به کار رود و هم به عنوان حرف الف استفاده شود. پس باید یک سری پیش پردازش روی کلمه ی فینگلیش انجام داد تا به درستی این کاراکترها با کاراکتر های کلمه ی غیرقانونی مقایسه شوند و حالات مختلف آنها (اینکه اعراب بودند یا معادل یک مصوت آمده اند) در نظر گرفته شود
</div>

In [ ]:
def all_formats_o(word,all,indx):
  all.add(word)
  for i in range(indx,len(word)):
    if word[i]=="o" or word[i]=="O":
      all_formats_o(word[:i]+"v"+word[i+1:],all,i+1)
      all_formats_o(word[:i]+word[i+1:],all,i)

  return all

<div dir='rtl' style='direction:rtl:'>
<h4> توابع  ENG_to_format , PRS_to_format </h4>
این توابع نیز پیش پردازش های لازم برای شناسایی کلمات فینگلیش را انجام میدهند. همانطور که گفته شد کلمات فارسی و فینگلیش در نگارش خود تفاوت هایی دارند که مقایسه ی آنها به راحتی و با مقایسه ی حرف به حرف ممکن نیست و باید تفاوت های آن ها را درنظر گرفت. حالات زیادی وجود دارد که در فینگلیش نوشته میشود ولی در فارسی نه (مثل اعراب) و حالات زیادی وجود دارد که در فارسی نوشته میشود ولی در انگلیسی نه! مانند ع اول کلمه (مثلا عامل =َamel)
به همین علت باید کلمه ی غیرقانونی که به فارسی نوشته شده و کلمه ی فینگلیش هردو به یک فرمتی برده شوند که قابلیت مقایسه باهم را داشته باشند. توابع زیر این کار را انجام میدهند.
مثال های دیگری از یکسان سازی فرمت که توابع زیر انجام میدهد  مانند مصوت های زیادی که معادل یک صدا هستند مثل ض ز ظ ذ ولی در انگلیسی همگی z نوشته میشود.
</div>

In [ ]:
def ENG_to_format(word):

  if word[0]=="e" or word[0]=="o":
    word="@"+word[1:]
  if word[0]=="a" or word[0]=="i" or word[0]=="o":
    word="@"+word
  if word[-1]=="e":
    word=word[:len(word)-1]+"h"

  word=word.replace("oo","v")
  
  word=word.replace("e","")
  word=word.replace("u","v")
  word=word.replace("i","y")
  word=word.replace("ee","y")

  all=set([word])
  all=all_formats_a(word,all,0)
  all_list=list(all)

  for x in all_list:
    all_formats_o(x,all,0)

  all_list=list(all)


  res=[]

  for x in all_list:
    res.append(x.replace("o",""))
  return(set(res))

In [ ]:
def PRS_to_format(word):

  res=""
  if word[0]=="ع" or word[0]=="آ" or word[0]=="ا":
    res+="@"

  for w in word:
    if w=="ا":
      res+="a"
    elif w=="ب":
      res+="b"
    elif w=="پ":
      res+="p"
    elif w=="ت" or w=="ط":
      res+="t"
    elif w=="ث" or w=="ص" or w=="س":
      res+="s"
    elif w=="ج":
      res+="j"
    elif w=="چ":
      res+="ch"
    elif w=="ح":
      res+="h"
    elif w=="خ":
      res+="kh"
    elif w=="د":
      res+="d"
    elif w=="ذ"or w=="ض"or w=="ز"or w=="ظ":
      res+="z"
    elif w=="ر":
      res+="r"
    elif w=="ژ":
      res+="zh"
    elif w=="ش":
      res+="sh"
    elif w=="ق"or w=="غ":
      res+="q"
    elif w=="ف":
      res+="f"
    elif w=="ک":
      res+="k"
    elif w=="گ":
      res+="g"
    elif w=="ل":
      res+="l"
    elif w=="م":
      res+="m"
    elif w=="ن":
      res+="n"
    elif w=="و":
      res+="v"
    elif w=="ه" or w=="ح":
      res+="h"
    elif w=="ی":
      res+="y"
  return res
  

In [ ]:
def check_FiEnglish(word,illegal):
  if PRS_to_format(illegal) in list(word):
    return True
  else:
    return False

<div dir='rtl' style='direction:rtl:'>
<h4> تابع detect_fingilish </h4>
این تابع مواردی که در آن کلمه‌ی غیرقانونی به شکل فینگیلش نوشته شده باشد را تشخیص می‌دهد. به عنوان مثال اگر برای کلمه‌ی غیرقانونی فیلترشکن داشته باشیم: filtershekan
در این تابع ابتدا کلمه ی غیرقانونی و توکن هردو به فرمت مورد نظر برده میشوند و با check_finglish مقایسه میشود که آیا توکن فینگلیش معادل غیرقانونی بوده است یا نه.
</div>

In [ ]:
def detect_fingilish(token, illegal_words):
  word=ENG_to_format(token)
  for ill in illegal_words:
    if check_FiEnglish(word,ill):
      return ill
  return None

<div dir='rtl' style='direction:rtl:'>
<h4> تابع detect_typo </h4>
این تابع برای تشخیص غلط تایپی با این قانون است که یک حرف از حروف کلمه ی غیرقانونی با یک کاراکتر اشتباه جایگزین شده باشد یا یک حرف پاک شده باشد. مثل فیلترشکن و فیلترشکم
</div>

In [ ]:
def detect_typo(token, illegal_words):
  for ill in illegal_words:
    if len(token)==len(ill) and sum(c1 != c2 for c1, c2 in zip(token, ill))==1:
      return ill
    elif len(token)==len(ill)-1:
      if sum(c1 != c2 for c1, c2 in zip("@"+token, ill))==1 or sum(c1 != c2 for c1, c2 in zip(token+"@", ill))==1:
          return ill
      for i in range(len(token)-1):
        if sum(c1 != c2 for c1, c2 in zip(token[:i+1]+"@"+token[i+1:], ill))==1:
          return ill
  return None

<div dir='rtl' style='direction:rtl:'>
<h4> تابع make_regex </h4>
این تابع کلمه ی غیر قانونی را دریافت کرده و یک regular expression برای آن مینویسد. این RE قرار است در ادامه حالت های مختلفی از نوشتن کلمه ی غیر قانونی را شامل شود که به شرح زیر هستند
</br>
1- حالات مصوت های هم صدا (درواقع غلط املایی) : صیگار یا ثیگار به جای سیگار
</br>
2- حالات حروف هم صدای انگلیسی به جای صدای فارسی مثل sیگار به جای سیگار</br>
3- حالات حروف غیر فارسی بین حروف کلمه مانند س%یگا$$ر</br>
4- هر ترکیبی از حالات فوق، مثلا صی&g#ار</br>
</div>

In [ ]:
def make_regex(word,gp):
  a="[a,A,ا,آ]"
  b="[b,B,ب]"
  p="[p,P,پ]"
  t="[t,T,ط,ت]"
  s="[s,S,c,S,س,ص,ث]"
  j="[j,J,g,G,ج]"
  ch="[ch|CH|cH|Ch|چ]"
  h="[h,H,ح,ه]"
  kh="[kh|kH|KH|Kh|خ]"
  d="[d|D|د]"
  z="[z|Z|ز|ض|ذ|ظ]"
  r="[r|R|ر]"
  zh="[zH|ZH|Zh|zh|ژ|j]"
  q="[q|Q|ق|غ]"
  g="[g|G|گ]"
  m="[m|M|م]"
  v="[v|W|w|V|u|U|و|o|O|oo|Oo|oO]"
  y="[i|I|Y|y|ی]"
  f="[f,F,ف]"
  l="[l,L,ل]"
  t="[t,T,ت,ط]"
  sh="[sh|ش|Sh|SH|sH]"
  k="[k,K,c,C,ک]"
  n="[n,N,ن]"


  res=""
  for w in word:
    if w=="آ" or w=="ا":
      res+=a+gp
    elif w=="ب":
      res+=b+gp
    elif w=="پ":
      res+=p+gp
    elif w=="ت" or w=="ط":
      res+=t+gp
    elif w=="ث" or w=="ص" or w=="س":
      res+=s+gp
    elif w=="ج":
      res+=j+gp
    elif w=="چ":
      res+=ch+gp
    elif w=="ح":
      res+=h+gp
    elif w=="خ":
      res+=kh+gp
    elif w=="د":
      res+=d+gp
    elif w=="ذ"or w=="ض"or w=="ز"or w=="ظ":
      res+=z+gp
    elif w=="ر":
      res+=r+gp
    elif w=="ژ":
      res+=zh+gp
    elif w=="ش":
      res+=sh+gp
    elif w=="ق"or w=="غ":
      res+=q+gp
    elif w=="ف":
      res+=f+gp
    elif w=="ک":
      res+=k+gp
    elif w=="گ":
      res+=g+gp
    elif w=="ل":
      res+=l+gp
    elif w=="م":
      res+=m+gp
    elif w=="ن":
      res+=n+gp
    elif w=="و":
      res+=v+gp
    elif w=="ه" or w=="ح":
      res+=h+gp
    elif w=="ی":
      res+=y+gp

  return res[:len(res)-len(gp)]


In [ ]:
def if_illegal_alter(txt,illegal):
  x = re.search(illegal, txt)
  if x:
    return True
  else:
    return False

<div dir='rtl' style='direction:rtl:'>
<h4> تابع detect_non_persian_alternative </h4>
این تابع توکن را با RE کلمه ی غیرقانونی که در بالا توضیح داده شد مقایسه میکند که اگر یکی از حالات توضیح داده شده رخ داده باشد توکن را به عنوان کلمه غیرقانونی معرفی میکند.
</div>

In [ ]:
def detect_non_persian_alternative(token, illegal_words):
  non_pers="[^پ,چ,ج,ح,خ,ه,ع,غ,ف,ق,ث,ص,ض,گ,ک,م,ن,ت,ا,ل,ب,ی,س,ش,و,ئ,د,ذ,ر,ز,ط,ظ,آ,ة,ي,ژ,ؤ,إ,أ,ء,ۀ]*"
  for ill in illegal_words:
    reg=make_regex(ill,non_pers)
    if if_illegal_alter(token,reg):
      return ill
  return None

<div dir='rtl' style='direction:rtl:'>
<h4> تابع detect_with_spaces </h4>
این تابع حالاتی را هندل میکند که بین کلمه ی غیرقانونی علاوه بر کاراکتر غیرفارسی فاصله باشد مثلا فی لتر شکن
</br>
نکته1: علت جدا کردن این تابع از RE قبلی این هست که در توکن کردن بر حسب فاصله بخش های مخلف کلمه ی جدا شده با اسپیس از هم جدا میشوند و باید یک تابع جدا برایش تعریف کرد
</br>
نکته2: این تابع حالات خاص جایگزینی مصوت ها را هم هندل میکند مثل صی gار
</div>

In [536]:
def detect_with_spaces(self,text,illegal_words):
  res=dict([])
  for ill in illegal_words:
    reg=make_regex(ill,"[^پ,چ,ج,ح,خ,ه,ع,غ,ف,ق,ث,ص,ض,گ,ک,م,ن,ت,ا,ل,ب,ی,س,ش,و,ئ,د,ذ,ر,ز,ط,ظ,آ,ة,ي,ژ,ؤ,إ,أ,ء,ۀ]*")
    for m in re.finditer(reg, text):
      mIdx=self.find_idx(text[m.start():m.end()],text,self.current_map)
      if not ill in res:
        res[ill]=set([mIdx])
      else:
        res[ill].add(mIdx)
      
  return res

<div dir='rtl' style='direction:rtl:'>
<h4> تابع detect_with_useless_persian_letters </h4>
این تابع حالاتی را هندل میکند که هر یک از حروف کلمه غیر قانونی، در جای خود تکرار داشته باشند.
بدین ترتیب امکان تشخیص "فیلللللترشکککککن"
برای کلمه غیر قانونی
'فیلترشکن'
وجود دارد.
</br>


In [601]:
def detect_with_useless_persian_letters(token = "سلااااام", illegal_words=['سلام','سلام']):

  regex = r"([^\W\d_])\1{1,}"
  subst = "\\1"

  for illegal_word in illegal_words:
    # You can manually specify the number of replacements by changing the 4th argument
    result = re.sub(regex, subst, token, 0, re.MULTILINE)
    # print(result, illegal_word)

    if result == illegal_word:
      return illegal_word

  return None

<div dir='rtl' style='direction:rtl:'>
<h4> تابع number_convertor </h4>
این تابع به کمک کتابخانه‌ی num2fawords اعداد را به شکل نوشتاری آن‌ها تبدیل می‌کند تا بعدا امکان تشخیص حالاتی مانند ۳۰گار بجای کلمه‌ی غیرقانونی سیگار وجود داشته باشد.
</div>

In [602]:
from __future__ import unicode_literals
from hazm import *
from num2fawords import words, ordinal_words

import string
def find_whitespace(st):
    for index, character in enumerate(st):
      if character in string.whitespace:
            yield index


def number_convertor(main_text = "چند عدد 30گار گرفتم"):

  main_text = main_text.replace('\u202F', ' ') # replace half spaces with regular spaces

  number_indexes = [] # indexes of all numbers in text
  number_lengths = [] # length of numbers in text
  new_whitespace_indices = []
  output_list = []
  """
  Takes a text which may contains a word as combination of number and subtext,
  then converts the 'number' part to its corrosponding text.

  the only input is a simple text.

  https://pypi.org/project/num2fawords/
  """

  output = ''

  normalizer = Normalizer()
  tokens = word_tokenize(normalizer.normalize(main_text))
  # print(tokens)

  whitespace_indices = list(find_whitespace(main_text))
  # print(whitespace_indices)

  temp = None

  for item in tokens:
    try:
      words(item)
      temp = words(item)
      if item == '.':
        temp = '.'
      number_lengths.append(len(temp))
      number_indexes.append(normalizer.normalize(main_text).index(item))
      index = normalizer.normalize(main_text).index(item)
      length = len(item)

      length_temp = len(temp)

      output_list.append([item, [index, index + length], [index, index + length_temp]])

    
    except: 
      if temp:
        output += temp+item+' '
        temp = None

      else:
        # pass
        output += item+' '
  
  # for space_position in whitespace_indices:
  #   if space_position > number_indexes[0]:
  #     space_position = space_position + number_lengths[0]

  # new_whitespace_indices.append(space_position)
  # print(number_indexes)
  # print(new_whitespace_indices)

  if output[-1] == ' ':
    output = output[:-1] + ''
  return output, output_list


<div dir='rtl' style='direction:rtl:'>
<h4> تابع emoji_convertor </h4>
این تابع به کمک دیتاستی از ایموجی ها که unicode 
متناظر با هریک از آن ها موجود است، به تبدیل ایموجی موجود در جمله به کلمه متناظر با آن می پردازد. 
بدین ترتیب امکان تشخیص "🐘ترشکن"
برای کلمه غیر قانونی
'فیلترشکن'
خواهد داشت.
</div>

In [600]:
def emoji_convertor(main_text="🐘ترشکن"):

  output = None
  temp_text = main_text

  output_list = []

  """
  Takes a text which may contains a word as combination of emoji and subtext,
  then converts the 'emoji' part to its corrosponding text.

  https://github.com/skorani/Preprocess-Emoji

  """

  # for colab
  try:
    import google.colab
    IN_COLAB = True

    !git clone https://github.com/skorani/Preprocess-Emoji.git
    !cp /content/Preprocess-Emoji/emojies_data.py /content/emojies_data.py
    !python /content/Preprocess-Emoji/emojies_data.py

  except:
    IN_COLAB = False
  
  from emojies_data import EMOJI_ALIAS_UNICODE
  
  main_unicode_code_points = [f"\\U{x:08x}" for x in map(ord, main_text)]
  main_unicode_text = "".join(main_unicode_code_points)

  for item in main_unicode_code_points:
    for k, v in EMOJI_ALIAS_UNICODE.items():

      text = v
      unicode_code_points = [f"\\U{x:08x}" for x in map(ord, text)]
      unicode_text = "".join(unicode_code_points)

      if unicode_text == item:
        # print(len((str(k[1:-1]))))  #string of emoji  
        # if item in main_unicode_code_points: #index of emoji
        #   print(main_unicode_code_points.index(item))

        # index = main_unicode_text.rfind(item)
        index = main_unicode_code_points.index(item)
        # Slice string to remove character at index 
        if len(main_text) > index:
          main_text = main_text[0 : index : ] + main_text[index + 1 : :]

        main_text = main_text[:index] + k[1:-1] + main_text[index:]
        length = len((str(k[1:-1])))
        emoji_word = str(k[1:-1])
        output = main_text

        output_list.append([emoji_word, [index, index+1], [index, index + length]])

  if output is None:
    output = temp_text

  # output_list.insert(0, output)
  return output, output_list
  
# print(emoji_convertor(main_text='🐘ترشکن سلام     *ف*گ غیرقانونی   و غیره 30گار خواهد شد.'))

<div dir='rtl' style='direction:rtl:'>
<h4> کلاس DetectIlligalWords </h4>
کلاس اصلی است که در ابتدای نوت‌بوک توضیح آن داده شد.
</div>

In [598]:
class DetectIlligalWords:
  def __init__(self, mode = 1):
    # currently there is no different mode
    self.mode = mode
    self.original_text = ""
    self.converted_text = ""
    self.emoji_idxes = []
    self.num_idxes = []
    self.current_map = []

  def preprocess(self, input, convertor = 'emoji'):
    self.original_text = input
    # normalize the input text
    normalizer = Normalizer()
    text = normalizer.normalize(input)
    self.normalized_text = text

    if convertor == 'emoji':
      #convert emojis
      text, self.emoji_idxes = emoji_convertor(text)
      self.converted_text = text
    elif convertor == 'number':
      text, self.num_idxes = number_convertor(text)
      self.converted_text = text

    # tokenize the normalized text
    tokens = word_tokenize(text)

    return tokens,text

  def find_idx(self, token, converted_text, idx_map):
    tmp_idx = converted_text.index(token)
    idx = [tmp_idx, tmp_idx + len(token) - 1]

    for i in idx_map:
      real = i[1]
      unreal = i[2]
      if idx[1] >= real[0]:
        idx[1] -= unreal[1] - real[1] 

      if idx[0] >= real[1]:
        idx[0] -= unreal[1] - real[1] 

    return tuple(idx)

  def merge_dics(self, dic1, dic2):
    keys = set(dic1.keys()).union(set(dic2.keys()))
    merged_dic = {}
    for key in keys:
      value = set()
      if key in dic1:
        value = value.union(dic1[key])
      if key in dic2:
        value = value.union(dic2[key])

      merged_dic[key] = value

    return merged_dic 

  def detect_funcs(self, token, illegal_words):
    illegal_word = None

    illegal_word = detect_with_useless_persian_letters(token, illegal_words)
    if illegal_word:
      # print('this is useless perisan')
      return illegal_word

    illegal_word = detect_non_persian_alternative(token, illegal_words)
    if illegal_word:
      # print('this is non persian alternative')
      return illegal_word

    illegal_word = detect_with_star(token, illegal_words)
    if illegal_word:
      # print('this is star')
      return illegal_word

    illegal_word = detect_fingilish(token, illegal_words)
    if illegal_word:
      # print('this is fingilish')
      return illegal_word


    illegal_word = detect_typo(token, illegal_words)
    if illegal_word:
      # print('this is typo')
      return illegal_word


  def detectIlligalWordsInTokens(self, all_tokens, illegal_words, text):
    detected_illegal_words = {}
    illegal_word = None
    tokens = [] # contains meaningless tokens
    illegal_tokens = {}


    for token in all_tokens:
      if token in illegal_words:
        # print('this is simple case')
        illegal_word = token

        if illegal_word:
          illegal_tokens[illegal_word] = set([token])
          illegal_word = None

      elif not is_meaningful_word(token): # removing meaningful words
        tokens.append(token)

    for token in tokens:
      illegal_word = self.detect_funcs(token, illegal_words)

      if illegal_word:
        if illegal_word not in illegal_tokens:
          illegal_tokens[illegal_word] = set([token])
 
        else:
          illegal_tokens[illegal_word].add(token)
        illegal_word = None

    for illegal_word, tokens in illegal_tokens.items():
      for token in tokens:
        idx = self.find_idx(token, self.converted_text, self.current_map)

        if illegal_word not in detected_illegal_words:
          detected_illegal_words[illegal_word] = set()
      
        detected_illegal_words[illegal_word].add(idx)

    return detected_illegal_words


  def detectIlligalWordsInText(self, text, illegal_words):
    detected_illegal_words = {}

    detected_illegal_words = detect_with_spaces(self, text, illegal_words)

    return detected_illegal_words


  def detect(self, tokens, illegal_words, text):
    detected_words_tokens = self.detectIlligalWordsInTokens(tokens, illegal_words, text)

    detected_words_text = self.detectIlligalWordsInText(text, illegal_words)

    detected_words = self.merge_dics(detected_words_tokens, detected_words_text) 

    return detected_words

  def process(self, tokens, illegal_words):
    detected_original_words = self.detect(tokens, illegal_words, self.converted_text)

    # lemmatization and detection
    lemmatizer = Lemmatizer()
    all_tokens_lemma = [lemmatizer.lemmatize(t) for t in tokens]
    detected_lemma_words = self.detect(tokens, illegal_words, self.converted_text)

    detected_words = self.merge_dics(detected_original_words, detected_lemma_words)

    return detected_words

  def run(self, input: str, illegal_words: list):
    # print('###------ with emoji --------###')
    tokens, text = self.preprocess(input, convertor = 'emoji')
    self.current_map = self.emoji_idxes
    detected_words_with_emoji = self.process(tokens, illegal_words)
    
    # print('###------ with number --------###')
    tokens, text = self.preprocess(input, convertor = 'number')
    self.current_map = self.num_idxes
    detected_words_with_number = self.process(tokens, illegal_words)

    detected_words = self.merge_dics(detected_words_with_emoji, detected_words_with_number)

    print("****************FINAL RESULT****************")
    print(detected_words)

<div dir='rtl' style='direction:rtl:'>
<h3> تست نهایی </h3>
در این قسمت خروجی تست نهایی کار برای حالت‌های مختلفی وجود دارد. به عنوان مثال با در نظر گرفتن کلمه‌ی چاقو به عنوان کلمه‌ی غیرمجاز، اگر در متن داشته باشیم 
"جا🦢" 
که یک اشتباه تایپی و یک استفاده از ایموجی در آن وجود دارد، می‌توانیم آن را تشخیص دهیم.
<br>
همچنین در حالت سخت‌تر و به عنوان یک متن کامل می‌توانیم متن زیر را در نظر بگیریم:
<br>
"این متن برای تست یافتن کلمات غیرقانونی مانند تفنگ و 🐘ترشکن و غیره استفاده خواهد شد. از دیگر موارد می‌توان به ت**گ و ۳۰گار و بmب و مثلا تقنگ و تف۵ن*گ و تف نگ و ب م ب اشاره کرد"
<br>
در این متن انواع و اقسام به کارگیری کلمات غیرقانونی آمده است که تمامی آن‌ها به همراه index آن‌ها تشخیص داده شده‌اند.
</div>

In [1]:
text = 'این متن برای تست یافتن کلمات غیرقانونی مانند تفنگ و 🐘ترشکن و غیره استفاده خواهد شد. از دیگر موارد می‌توان به ت**گ و ۳۰گار و بmب و مثلا تقنگ و تف۵ن*گ و تف نگ و ب م ب اشاره کرد'
# text = 'سلام 🐘ترشکن'
# text = 'سلام ۳۰ گا#ر'
# text = 'می‌تونی ب م ب رو تشخیص بدی؟'
# text = 'sigar بmب '
# text = 'جا🦢'
# text = 'تفگ'
# text = 'چااااقو'
# text = 'چ.ا.ق.و'
# text = 'filtershekan'

illegalWords = ['تفنگ','فیلترشکن','چاقو','سیگار','بمب']

detectIlligalWords = DetectIlligalWords()
detectIlligalWords.run(text,illegalWords)

NameError: name 'DetectIlligalWords' is not defined